<a href="https://colab.research.google.com/github/AtharvaGh/StartingOne/blob/main/Hyper_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U keras-Tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 6.6 MB/s 


In [ ]:
%tensorflow_version 2.0
import tensorflow as tf
device_name =tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU not found')
print('Found GPU at:{}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at:/device:GPU:0


In [ ]:
!pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from tensorflow import keras
from keras import layers
from keras_tuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split

In [ ]:
def build_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int('num_layers',2,20)):
    model.add(layers.Dense(units=hp.Int('unit_'+str(i),min_value=32,max_value=512,step=32),activation='relu'))
    model.add(layers.Dense(1,activation='linear'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('laerning_rate',[1e-2,1e-3,1e-4])),loss='mean_absolute_error',metrics=['mean_absolute_error'])
  return model

In [ ]:
power_df = pd.read_csv("/content/drive/MyDrive/Datasets/Tetuan City power consumption.csv")
X = power_df.iloc[:,2:4]
Y = power_df.iloc[:,0]
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,)

In [ ]:
tuner = RandomSearch(build_model,objective='val_mean_absolute_error',max_trials=5,executions_per_trial=3,directory='project',project_name='Power_Consumption')

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
unit_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
laerning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
unit_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}


In [ ]:
tuner.search(X_train,Y_train,epochs = 10,validation_data = (X_test,Y_test))

Trial 5 Complete [00h 05m 13s]
val_mean_absolute_error: 3.4967902501424155

Best val_mean_absolute_error So Far: 3.4967902501424155
Total elapsed time: 00h 20m 45s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in project/Power_Consumption
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 16
unit_0: 352
laerning_rate: 0.0001
unit_1: 160
unit_2: 288
unit_3: 512
unit_4: 256
unit_5: 320
unit_6: 416
unit_7: 512
unit_8: 480
unit_9: 256
unit_10: 320
unit_11: 96
unit_12: 224
unit_13: 96
unit_14: 448
unit_15: 128
unit_16: 384
Score: 3.4967902501424155
Trial summary
Hyperparameters:
num_layers: 9
unit_0: 224
laerning_rate: 0.001
unit_1: 256
unit_2: 32
unit_3: 32
unit_4: 32
unit_5: 32
unit_6: 32
unit_7: 32
unit_8: 32
Score: 3.552664279937744
Trial summary
Hyperparameters:
num_layers: 6
unit_0: 416
laerning_rate: 0.01
unit_1: 416
unit_2: 416
unit_3: 416
unit_4: 224
unit_5: 64
unit_6: 192
unit_7: 480
unit_8: 64
Score: 3.556450049082438
Trial summary
Hyperparameters:
num_layers: 5
unit_0: 160
laerning_rate: 0.0001
unit_1: 160
unit_2: 480
unit_3: 288
unit_4: 256
unit_5: 288
unit_6: 384
unit_7: 480
unit_8: 384
Score: 3.6084352334340415
Trial summary
Hyperparameters:
n

Classification problem with gridsearch

In [ ]:
churn_df = pd.read_csv("/content/drive/MyDrive/Datasets/Churn_Modelling(1).csv")
churn_df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:

#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X_train = sc.fit_trainform(X_train)
#X_test = sc.transform(X_test)

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense,Flatten,Activation,ReLU,LeakyReLU

In [ ]:
def create_model(layers,activation):
  model = Sequential()
  for i,nodes in enumerate(layers):
    if i == 0:
      model.add(Dense(nodes,input_dim=X_train.shape[1]))
      model.add(Activation(activation))
    else:
      model.add(Dense(nodes))
      model.add(Activation(activation))
  model.add(Dense(1))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model
model = KerasClassifier(build_fn=create_model,verbose=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  del sys.path[0]


In [ ]:
layers=[[20],[40,20],[45,30,15]]
activations = ['sigmoid','relu']
param_grid = dict(layers = layers,activation = activations,batch_size=[128,256],epochs=[10])
grid = GridSearchCV(estimator=model,param_grid=param_grid)

In [ ]:
grid_result = grid.fit(X_train,Y_train)

In [ ]:
[grid_result.best_score_,grid_result.best_params_]